In [1]:
import numpy as np
import pickle as pkl
import networkx as nx
import scipy.sparse as sp

from sklearn.model_selection import train_test_split
# from utils.utilities import run_random_walks_n2v

In [2]:
def load_graphs():
    """Load graph snapshots given the name of dataset"""
    with open("data/graphs/graph.pkl", "rb") as f:
        graphs = pkl.load(f)  # 16个时刻的图，节点，边的信息，节点的特征;
    print("Loaded {} graphs ".format(len(graphs)))
    adjs = [nx.adjacency_matrix(g) for g in graphs]  # 每个图的邻接矩阵
    return graphs, adjs

graphs, adjs = load_graphs()

Loaded 365 graphs 


In [3]:
graphs[0].graph['feature'].A # 可以与2里dataframe对照检查一下

array([[28.92, 28.94, 28.96, ..., 28.78, 28.8 , 28.81],
       [ 0.  ,  0.  ,  0.  , ...,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  , ...,  0.  ,  0.  ,  0.  ],
       ...,
       [ 0.  ,  0.  ,  0.  , ...,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  , ...,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  , ...,  0.  ,  0.  ,  0.  ]])

In [4]:
graphs[364].graph['feature'].A 

array([[28.63, 28.64, 28.65, ..., 28.58, 28.58, 28.63],
       [ 0.  ,  0.  ,  0.  , ...,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  , ...,  0.  ,  0.  ,  0.  ],
       ...,
       [ 0.  ,  0.  ,  0.  , ...,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  , ...,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  , ...,  0.  ,  0.  ,  0.  ]])

In [5]:
feats = []
for i in range(len(graphs)):
    feats.append(graphs[i].graph['feature'])

len(feats)

365

In [6]:
feats[0].A

array([[28.92, 28.94, 28.96, ..., 28.78, 28.8 , 28.81],
       [ 0.  ,  0.  ,  0.  , ...,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  , ...,  0.  ,  0.  ,  0.  ],
       ...,
       [ 0.  ,  0.  ,  0.  , ...,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  , ...,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  , ...,  0.  ,  0.  ,  0.  ]])

In [7]:
feats[364].A

array([[28.63, 28.64, 28.65, ..., 28.58, 28.58, 28.63],
       [ 0.  ,  0.  ,  0.  , ...,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  , ...,  0.  ,  0.  ,  0.  ],
       ...,
       [ 0.  ,  0.  ,  0.  , ...,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  , ...,  0.  ,  0.  ,  0.  ],
       [ 0.  ,  0.  ,  0.  , ...,  0.  ,  0.  ,  0.  ]])

In [8]:
print(len(graphs), len(adjs))

365 365


In [9]:
def get_context_pairs(graphs, adjs):
    """ Load/generate context pairs for each snapshot through random walk sampling."""
    print("Computing training pairs ...")
    context_pairs_train = []
    # 在每一个时间步骤，进行一次随机游走
    for i in range(len(graphs)):
        context_pairs_train.append(run_random_walks_n2v(graphs[i], adjs[i], num_walks=10, walk_len=20))

    return context_pairs_train # 在16个时间步都获得了每个节点对应的上下文节点
def run_random_walks_n2v(graph, adj, num_walks, walk_len):
    """ In: Graph and list of nodes
        Out: (target, context) pairs from random walk sampling using 
        the sampling strategy of node2vec (deepwalk)"""
    nx_G = nx.Graph()
    for e in graph.edges():
        nx_G.add_edge(e[0], e[1])  # 添加节点; 重复只保留一次; 只将有连接的节点保留，没有连接的去掉（因为在构图时，将之前的节点加入了）
    for edge in graph.edges():
        nx_G[edge[0]][edge[1]]['weight'] = adj[edge[0], edge[1]]  # 连接的边的权重; 也就是连接的次数

    # 经过以上操作，这个图包括节点，边以及边的权重
    # 开始定义node2vec随机游走
    G = Graph_RandomWalk(nx_G, False, 1.0, 1.0)
    G.preprocess_transition_probs()  # 游走概率计算
    walks = G.simulate_walks(num_walks, walk_len)  # 随机游走 18(点)*10(num_walks)个采样序列，每个20步
    WINDOW_SIZE = 10
    pairs = defaultdict(list)
    pairs_cnt = 0
    for walk in walks:
        for word_index, word in enumerate(walk):
            for nb_word in walk[max(word_index - WINDOW_SIZE, 0): min(word_index + WINDOW_SIZE, len(walk)) + 1]:  # 该节点的window上下文节点
                if nb_word != word:
                    pairs[word].append(nb_word)  # 和本身节点不同，则作为一对训练语料
                    pairs_cnt += 1
    print("# nodes with random walk samples: {}".format(len(pairs)))
    print("# sampled pairs: {}".format(pairs_cnt))
    return pairs
# node2vec的训练语料; 365个garph 和 365个节点特征;

time_steps =365
assert time_steps <= len(adjs), "Time steps is illegal"
context_pairs_train = get_context_pairs(graphs, adjs)  # 16个图，每个图中进行随机游走采样;

Computing training pairs ...


IndexError: column index (782) out of range

In [34]:
for e in graphs[0].edges():
    print(e[0], e[1])

1 347


: 